In [1]:
# optical flow Vis
import numpy as np
from matplotlib import colors
import imageio
from pathlib import Path
from PIL import Image

def flow_16bit_to_float(flow_16bit: np.ndarray):
    assert flow_16bit.dtype == np.uint16
    assert flow_16bit.ndim == 3
    h, w, c = flow_16bit.shape
    assert c == 3

    valid2D = flow_16bit[..., 2] == 1
    assert valid2D.shape == (h, w)
    assert np.all(flow_16bit[~valid2D, -1] == 0)
    valid_map = np.where(valid2D)

    # to actually compute something useful:
    flow_16bit = flow_16bit.astype('float')

    flow_map = np.zeros((h, w, 2))
    flow_map[valid_map[0], valid_map[1], 0] = (flow_16bit[valid_map[0], valid_map[1], 0] - 2 ** 15) / 128
    flow_map[valid_map[0], valid_map[1], 1] = (flow_16bit[valid_map[0], valid_map[1], 1] - 2 ** 15) / 128
    return flow_map, valid2D

def load_flow(flowfile: str):
    flowfile = Path(flowfile)
    assert flowfile.exists()
    assert flowfile.suffix == '.png'
    flow_16bit = imageio.imread(str(flowfile), format='PNG-FI')
    flow, valid2D = flow_16bit_to_float(flow_16bit)
    return flow, valid2D

def visualize_optical_flow(flow, scaling=None):
    # flow -> numpy array 2 x height x width
    # 2,h,w -> h,w,2
    flow = flow.transpose(1,2,0)
    flow[np.isinf(flow)]=0
    # Use Hue, Saturation, Value colour model
    hsv = np.zeros((flow.shape[0], flow.shape[1], 3), dtype=float)

    # The additional **0.5 is a scaling factor
    mag = np.sqrt(flow[...,0]**2+flow[...,1]**2)**0.5

    ang = np.arctan2(flow[...,1], flow[...,0])
    ang[ang<0]+=np.pi*2
    hsv[..., 0] = ang/np.pi/2.0 # Scale from 0..1
    hsv[..., 1] = 1
    if scaling is None:
        hsv[..., 2] = (mag-mag.min())/(mag-mag.min()).max() # Scale from 0..1
    else:
        mag[mag>scaling]=scaling
        hsv[...,2] = mag/scaling
    rgb = colors.hsv_to_rgb(hsv)
    bgr = np.stack([rgb[...,2],rgb[...,1],rgb[...,0]], axis=2)
    return bgr

flow, valid2D = load_flow('D:/Game/000128.png')
rgb_flow = visualize_optical_flow(np.transpose(flow, (2, 0, 1)))  # (480, 640, 3) 0~1
pil_rgb_flow = Image.fromarray(np.uint8(rgb_flow * 255))
pil_rgb_flow.save('D:/Game/000128_rgb.png')
pil_rgb_flow.show('1')

In [25]:
import torch
from PIL import Image

a = Image.open(r'G:\DSEC_Dataset\train_images\interlaken_00_c\images\left\rectified\000000.png')
b = Image.open(r'G:\DSEC_Dataset\train_images\interlaken_00_c\images\left\rectified\000010.png')


images_list = [np.array(a), np.array(b)]
h, w, c = images_list[0].shape
image_show = np.zeros((h, 0, c))

for image in images_list:
    image_show = np.concatenate((image_show, image), axis=1)
Image.fromarray(np.uint8(image_show)).show('1')


In [4]:
a = [1, 2, 3, 4, 5, 6, 7]
print(sum(a[-3:]) / 3)

6.0
